# Ejercicio de Localización
## V2 - Modelo generico de Localización
### En cada región existe un mercado consumidor con una demanda conocida y es posible instalar palntas productoras chicas o grandes

In [36]:
# Importación de librerias
# OR Tools - Incluye resolución de programación lineal
from ortools.linear_solver import pywraplp
from tabulate import tabulate

# ortools setup
import pandas as pd
import matplotlib.pyplot as plt
import csv
import os
    

## Inputs para la resolución del problema

In [37]:
# region  Regiones donde opera --> Id Region: Nombre Regrion
region = { 
 (0):'Nort America',
 (1):'Sur America',
 (2):'Europa',
 (3):'Aaia',
 (4):'Africa'
}

#  demanda --> Demanda estimada de cada rgión --> Id Region: Demanda
demanda = {
 (0):12,
 (1):8,
 (2):14,
 (3):16,
 (4):7
}

# planta Locaciones posibles de donde fabricar [Id Fuente, Id Region, Tamaño , Costo Fijo, Capaciad]

planta = [
    [0,0, 'C', 6000, 10],
    [1,0, 'G', 9000, 20],
    [2,1, 'C', 4500, 10],
    [3,1, 'G', 6750, 20],
    [4,2, 'C', 6500, 10],
    [5,2, 'G', 9750, 20],
    [6,3, 'C', 4100, 10],
    [7,3, 'G', 6150, 20],
    [8,4, 'C', 4000, 10],
    [9,4, 'G', 6000, 20]
]

# costo  Costo de fabricar en la region 'x' y vender en la región 'y' siendo 'x' y 'y' las regiones donde opera limpiamax.

costo = {
    (0,0):81,
    (0,1):92,
    (0,2):101,
    (0,3):130,
    (0,4):115,
    (1,0):117,
    (1,1):77,
    (1,2):108,
    (1,3):98,
    (1,4):100,
    (2,0):102,
    (2,1):105,
    (2,2):95,
    (2,3):119,
    (2,4):111,
    (3,0):115,
    (3,1):125,
    (3,2):90,
    (3,3):59,
    (3,4):74,
    (4,0):142,
    (4,1):100,
    (4,2):103,
    (4,3):105,
    (4,4):71
}

Mostrar Datos:

In [42]:

print("Listado de regiones y demandas:\n")
#for i in range(len(region)):
#    nombre = region[i]
#    demanda = demanda.get(i, "No disponible")
#    print(f"{i}. Región: {region[i]} - Demanda: {demanda[i]}")

tabla = []
for i in range(len(region)):
    campo_1 = region[i]
    campo_2 = demanda.get(i, "No disponible")
    fila = [i, campo_1, campo_2]
    tabla.append(fila)

# Mostrar la tabla
print(tabulate(tabla, headers=["#", "Región", "Demanda"], tablefmt="grid"))

# Mostrar matriz de costo
matriz_costos = []
for i in range(len(region)):
    line_matriz = []
    line_matriz.append(region[i])
    for j in range(len(region)):
        valor = costo.get((i,j),0)
        line_matriz.append(valor)
    matriz_costos.append(line_matriz)

#print("\n")
print("Matriz de Costos de producir en la planta n y entregar en el mercado m:\n")
#for row in matriz_costos:
#    print(''.join(f"{elem:6}" for elem in row))

# Mostrar la tabla
header = []
header.append('Planta/Mercado')
for i in range(len(region)):
    header.append(region[i])
print(tabulate(matriz_costos, headers = header,  tablefmt="grid"))


Listado de regiones y demandas:

+-----+--------------+-----------+
|   # | Región       |   Demanda |
+=====+==============+===========+
|   0 | Nort America |        12 |
+-----+--------------+-----------+
|   1 | Sur America  |         8 |
+-----+--------------+-----------+
|   2 | Europa       |        14 |
+-----+--------------+-----------+
|   3 | Aaia         |        16 |
+-----+--------------+-----------+
|   4 | Africa       |         7 |
+-----+--------------+-----------+
Matriz de Costos de producir en la planta n y entregar en el mercado m:

+------------------+----------------+---------------+----------+--------+----------+
| Planta/Mercado   |   Nort America |   Sur America |   Europa |   Aaia |   Africa |
+==================+================+===============+==========+========+==========+
| Nort America     |             81 |            92 |      101 |    130 |      115 |
+------------------+----------------+---------------+----------+--------+----------+
| Sur America 

## Planteo del problema de Programación Lineal

In [39]:
# Incognitas
# x es la cantidad que voy a fabricar en la planta 'p' para abastecer el mercado 'm', asumo valores enteros positivos
# y es una variable que me indica si la planta debe ser puesta en marcha asume valores boleanos 0,1
x = {}
y = {}

# Create the mip solver with the SCIP backend.  Integer Linear Programing
solver = pywraplp.Solver.CreateSolver('SCIP')

infinity = solver.infinity()

for n in range(len(region)):
    for m in range(len(region)):
        x[(n),(m)] = solver.IntVar(0.0, infinity, 'x: F. '+region.get(n)+'-'+' R. '+region.get(m))
#        print (x[(p),(m)])

for p in range(len(planta)):
        y[(p)] = solver.BoolVar('y: Planta en: '+region.get(planta[p][1])+'-'+' Tamaño: '+ planta[p][2])
#        print (y[(p)])

# Ecuaciones para controlar el abastecimiento - 
# La suma de las x fabricadas en los distintos mercados origen para abastecer el mercado 'm' destino debe ser = a la demanda
for m in range(len(region)):
    produccion_d = []
    for n in range(len(region)):
        produccion_d.append(x[(n),(m)])
#        print (produccion_d)
    suma_produccion_d = sum(produccion_d)
    solver.Add(suma_produccion_d == demanda.get(m))
    
# Ecuaciones para controlar la fabricación
# La cantidad que se desea fabricar en cada mercado no debe sobreásar su capacidad de fabricación 
for n in range(len(region)):
    produccion_p = []
    capacidad = []
    for m in range(len(region)):
        produccion_p.append(x[(n),(m)])
    suma_produccion_p = sum(produccion_p)
    for p in range(len(planta)):
        if planta[p][1] == n:
            capacidad.append(planta[p][4] * y[(p)])
    suma_capacidad = sum(capacidad)
    solver.Add(suma_produccion_p <= suma_capacidad)

# Función de Costos
# Es la suma de las cantidades que se fabrican en una planta con destino a una región más el costo fijo de dicha planta
costo_produccion = []
costo_fijo = []
for n in range(len(region)):
    for m in range(len(region)):
        costo_produccion.append(x[(n),(m)] * costo.get((n,m),0))
#        print (f"{x[(p),(m)]} por {costo.get((p,m))}")
    
for p in range(len(planta)):
    costo_fijo.append(y[(p)] * planta[p][3])

suma_costo_produccion = sum(costo_produccion)
suma_costo_fijo = sum(costo_fijo)
solver.Minimize(suma_costo_produccion + suma_costo_fijo )



## Resolución

In [40]:
# Ejecusión del programa
status = solver.Solve()

## Resultados

In [41]:
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value()) 
    print('\nPlantas a Utilizar:')
    
    for p in range(len(planta)):
        if y[(p)].solution_value() == 1:
            print (f"{y[(p)]} = {y[(p)].solution_value()}")

    # Mostrar matriz de costo
    matriz_alocacion = []
    for n in range(len(region)):
        line_matriz = []
        line_matriz.append(region[n])
        valor_total = 0
        capacidad_total = 0
        for m in range(len(region)):
            valor = x[(n),(m)].solution_value()
            line_matriz.append(valor)
            valor_total = valor_total + valor
        line_matriz.append(valor_total)
        for p in range(len(planta)):
            if planta[p][1] == n:
                capacidad_total = capacidad_total +  (planta[p][4] * y[(p)].solution_value()) ## ##*
        line_matriz.append(capacidad_total)
        line_matriz.append(capacidad_total - valor_total)

        matriz_alocacion.append(line_matriz)
    
    line_matriz = []
    line_matriz.append('Producción')
    for m in range(len(region)):
        valor_total_m = 0
        for n in range(len(region)):
            valor_total_m = valor_total_m + x[(n),(m)].solution_value()
        line_matriz.append(valor_total_m)
    matriz_alocacion.append(line_matriz)
    line_matriz = []
    line_matriz.append('Demanda')
    for m in range(len(region)):
        line_matriz.append(demanda.get(m,0))
    matriz_alocacion.append(line_matriz)

    #print("\n")
    print("Matriz de Alocación de producir en la planta n y entregar en el mercado m:\n")
    #for row in matriz_costos:
    #    print(''.join(f"{elem:6}" for elem in row))

    # Mostrar la tabla
    header = []
    header.append('Planta/Mercado')
    for i in range(len(region)):
        header.append(region[i])
    header.append('Producción')
    header.append('Capacidad')
    header.append('Balance')
    print(tabulate(matriz_alocacion, headers = header,  tablefmt="grid"))


else:
    print('The problem does not have an optimal solution.')
    
print('Problem solved in %f milliseconds' % solver.wall_time())
print('Problem solved in %d iterations' % solver.iterations())
print('Problem solved in %d branch-and-bound nodes' % solver.nodes())


Solution:
Objective value = 23751.0

Plantas a Utilizar:
y: Planta en: Sur America- Tamaño: G = 1.0
y: Planta en: Aaia- Tamaño: G = 1.0
y: Planta en: Africa- Tamaño: G = 1.0
Matriz de Alocación de producir en la planta n y entregar en el mercado m:

+------------------+----------------+---------------+----------+--------+----------+--------------+-------------+-----------+
| Planta/Mercado   |   Nort America |   Sur America |   Europa |   Aaia |   Africa |   Producción |   Capacidad |   Balance |
+==================+================+===============+==========+========+==========+==============+=============+===========+
| Nort America     |              0 |             0 |        0 |      0 |        0 |            0 |           0 |         0 |
+------------------+----------------+---------------+----------+--------+----------+--------------+-------------+-----------+
| Sur America      |             12 |             8 |        0 |      0 |        0 |           20 |          20 |       